In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/dataset.yaml
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034602.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034892.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034730.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034468.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/035097.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034518.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034896.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034831.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/035905.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/034701.txt
/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs/labels/val/035816.txt
/kaggle/input

In [2]:
import os
import yaml

# Define the original dataset root (Kaggle attached dataset)
original_dataset_root = "/kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs"
original_yaml_file = os.path.join(original_dataset_root, "dataset.yaml")

# Load the original YAML file
with open(original_yaml_file, "r") as f:
    config = yaml.safe_load(f)

config["path"] = original_dataset_root


for split in ["train", "val", "test"]:
    if split in config:
        for key in config[split]:
            config[split][key] = config[split][key].replace("\\", "/")

# Write the updated YAML to a new temporary file in the writable working directory
temp_yaml_file = "/kaggle/working/dataset_temp.yaml"
with open(temp_yaml_file, "w") as f:
    yaml.dump(config, f)
    
print("Updated YAML configuration written to:", temp_yaml_file)
print(yaml.dump(config, default_flow_style=False))


Updated YAML configuration written to: /kaggle/working/dataset_temp.yaml
names:
  0: short_sleeve_top
  1: long_sleeve_top
  2: short_sleeve_outwear
  3: long_sleeve_outwear
  4: vest
  5: sling
  6: shorts
  7: trousers
  8: skirt
  9: short_sleeve_dress
  10: long_sleeve_dress
  11: vest_dress
  12: sling_dress
nc: 13
path: /kaggle/input/deepfashion2-10k-imgs/deepfashion-for-10k-imgs
test:
  images: test/image
  labels: test/labels
train:
  images: train/image
  labels: train/labels
val:
  images: val/image
  labels: val/labels



In [4]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.5/921.5 kB 16.1 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import torch
import os
from ultralytics import YOLO

# Define paths
DATA_YAML = "/kaggle/working/dataset_temp.yaml"
SAVE_DIR = "/kaggle/working/yolo_results"

# Load YOLOv8m-seg model
model = YOLO("yolov8m-seg.pt")

# Fine-tune on dataset
model.train(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,               # Increased from 8
    workers=4,              # Increased from 2
    project=SAVE_DIR,
    name="yolo_finetuned_v3",
    amp=True,
    half=True,
)

# Save best model path
BEST_MODEL_PATH = os.path.join(SAVE_DIR, "yolo_finetuned_v3", "weights", "best.pt")
print(f"Best model saved at: {BEST_MODEL_PATH}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 52.4M/52.4M [00:01<00:00, 32.8MB/s]


Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=segment, mode=train, model=yolov8m-seg.pt, data=/kaggle/working/dataset_temp.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=4, project=/kaggle/working/yolo_results, name=yolo_finetuned_v3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, sa

100%|██████████| 755k/755k [00:00<00:00, 4.55MB/s]


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytic

...

                   all      10001      10001       0.53      0.517      0.531      0.382      0.317      0.288      0.244     0.0854

50 epochs completed in 9.000 hours.
Optimizer stripped from /kaggle/working/yolo_results/yolo_finetuned_v3/weights/last.pt, 54.8MB
Optimizer stripped from /kaggle/working/yolo_results/yolo_finetuned_v3/weights/best.pt, 54.8MB

Validating /kaggle/working/yolo_results/yolo_finetuned_v3/weights/best.pt...
Ultralytics 8.3.78 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8m-seg summary (fused): 105 layers, 27,229,911 parameters, 0 gradients, 110.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 313/313 [02:24<00:00,  2.17it/s]


                   all      10001      10001      0.531      0.516      0.531      0.382      0.317      0.287      0.244     0.0855
      short_sleeve_top       1533       1533      0.531      0.556      0.592      0.458      0.471      0.389       0.36      0.131
       long_sleeve_top        877        877      0.524       0.53      0.531      0.389      0.433      0.344        0.3      0.104
  short_sleeve_outwear          6          6       0.74      0.333      0.449       0.22          0          0      0.146      0.016
   long_sleeve_outwear        163        163      0.485      0.295      0.369      0.275      0.449      0.205      0.192       0.08
                  vest        275        275      0.406      0.451      0.415      0.325      0.355      0.305       0.25     0.0743
                 sling         40         40      0.526      0.775      0.676      0.536      0.523       0.65      0.572      0.267
                shorts       2001       2001      0.561      0.806   

/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.1ms preprocess, 9.9ms inference, 0.0ms loss, 0.7ms postprocess per image
Results saved to /kaggle/working/yolo_results/yolo_finetuned_v3
Best model saved at: /kaggle/working/yolo_results/yolo_finetuned_v3/weights/best.pt


In [3]:
import os
print(os.listdir("/kaggle/working/yolo_results/yolo_finetuned_v3/weights"))


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/yolo_results/yolo_finetuned_v3/weights'